In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

import deepxde as dde
from deepxde.backend import tf

In [2]:
C = tf.Variable(2.0)

def pde(x, y):
    dy_x = tf.gradients(y, x)[0]
    dy_x, dy_t = dy_x[:, 0:1], dy_x[:, 1:]
    dy_xx = tf.gradients(dy_x, x)[0][:, 0:1]
    return (
        dy_t
        - C * dy_xx
        + tf.exp(-x[:, 1:])
        * (tf.sin(np.pi * x[:, 0:1]) - np.pi ** 2 * tf.sin(np.pi * x[:, 0:1]))
    )

def func(x):
    return np.sin(np.pi * x[:, 0:1]) * np.exp(-x[:, 1:])

geom = dde.geometry.Interval(-1, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

bc = dde.DirichletBC(geomtime, func, lambda _, on_boundary: on_boundary)
ic = dde.IC(geomtime, func, lambda _, on_initial: on_initial)

observe_x = np.vstack((np.linspace(-1, 1, num=10), np.full((10), 1))).T
ptset = dde.bc.PointSet(observe_x)
observe_y = dde.DirichletBC(
    geomtime, ptset.values_to_func(func(observe_x)), lambda x, _: ptset.inside(x)
    )

data = dde.data.TimePDE(
        geomtime,
        pde,
        [bc, ic, observe_y],
        num_domain=40,
        num_boundary=20,
        num_initial=10,
        anchors=observe_x,
        solution=func,
        num_test=10000,
    )

In [6]:
layer_size = [2] + [32] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)

model.compile("adam", lr=0.001, metrics=["l2 relative error"])
variable = dde.callbacks.VariableValue(C, period=1000)
losshistory, train_state = model.train(epochs=50000, callbacks=[variable])

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

W0814 16:25:05.943086 47161070172352 deprecation.py:323] From /u/wangnisn/.local/lib/python3.7/site-packages/deepxde/maps/fnn.py:82: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.
W0814 16:25:05.952180 47161070172352 deprecation.py:323] From /mpcdf/soft/SLE_15/packages/skylake/tensorflow/gpu/anaconda_3_2020.02-2020.02-cuda_10.2-10.2.89/2.2.0/lib/python3.7/site-packages/tensorflow/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


Compiling model...
Building feed-forward neural network...
'build' took 0.054246 s

'compile' took 1.492338 s

Initializing variables...
Training model...

Step      Train loss                                  Test loss                                   Test metric   
0         [1.66e+01, 1.41e-01, 9.21e-01, 2.06e-01]    [1.86e+01, 0.00e+00, 0.00e+00, 0.00e+00]    [1.40e+00]    
0 2.0
1000      [7.72e-03, 2.82e-02, 5.23e-02, 1.56e-02]    [1.13e-02, 0.00e+00, 0.00e+00, 0.00e+00]    [3.89e-01]    
1000 1.98e+00
2000      [1.67e-03, 6.53e-03, 9.45e-03, 7.51e-03]    [5.64e-03, 0.00e+00, 0.00e+00, 0.00e+00]    [4.13e-01]    
2000 1.68e+00
3000      [8.49e-04, 1.91e-03, 1.36e-03, 4.07e-03]    [4.83e-03, 0.00e+00, 0.00e+00, 0.00e+00]    [2.99e-01]    
3000 1.43e+00
4000      [2.85e-04, 7.09e-04, 1.21e-04, 1.35e-03]    [3.16e-03, 0.00e+00, 0.00e+00, 0.00e+00]    [1.66e-01]    
4000 1.21e+00
5000      [1.70e-04, 2.74e-04, 1.84e-05, 2.66e-04]    [1.52e-03, 0.00e+00, 0.00e+00, 0.00e+00]    [7.34e

KeyboardInterrupt: 

In [ ]:
main()